# Практика 2. Вывод модели в Production

## Повторение пройденного:

### 1. Мы используем обученную модель повторно. Если обучение прошло успешно, мы хотим сохранить ее и использовать много раз. Когда возникнет необходимость, мы ее переобучим и сохраним для последующего использования и так - до бесконечности.
### 2. В нашем распоряжении есть несколько средств:
#### 2.1. Сериализовать нашу модель и объекты с нужными нам данными в файл с использованием Pickle
#### 2.2. Сериализовать нашу модель и пайплайн обработки в файл с помощью joblib
#### 2.3. Сохранить обученную модель в формате ONNX (Open Neural Network Exchange) в файл
### 3. Эти форматы позволяют нам сохранить результат обучения для повторного использования или передаче работоспособной модели нашим коллегам.

## Что мы сегодня делаем?

### У нас есть датасет, где собрана информаци по тендерам в файле CSV (tenders.csv в папке X в ваших блокнотах Collab). В датасете такие поля, которые мы будем использовать или нет:

#### Name - название тендера (в идеале - описание предмета закупки). Текст, который мы будем анализировать и искать соответствие его тому, что мы предполагаем увидеть в классификаторе - PurchaseArea, см. ниже.
#### Id - номер тендера. Исключительно техническая нагрузка. Уникальный идентификатор строки.
#### Region - Информационно. Наименование региона. Например, "Орловская обл."
#### Location - Информационно. Название города , куда осуществляется закупка. Например, "г. Усть-Баргузин"
#### Buyer - Информационно. Наименование организации, объявившей закупку.
#### TenderType - Классификатор торговой процедуры: 44-ФЗ, 223-ФЗ, Коммерческая. Будет использоваться для вспомогательной аналитики.
#### Placement - Способ размещения торговой процедуры: Электронные аукционы, Запросы котировок и предложений, Нерегламентированные закупки и т.п. Будет использоваться для вспомогательной аналитики
#### PublishedDate - Информационно. Дата публикации
#### StartingPrice - Начальная (минимальная) цена торгов. Отобраны только непустые значения с объявленной ценой. Будем использовать для вспомогательного анализа.
#### PurchaseArea - Классификатор вида закупки. Мы это будем пытаться предсказывать.

### Этот датасет мы преобразуем в DataFrame и используем для обучения модели.

## Ниже проиллюстрированы шаги работы. ВЫ ДОЛЖНЫ МОДИФИЦИРОВАТЬ подготовку данных и обучение модели. Результат сохранить в перечисленные форматы экспорта.

## Ваши товарищи возьмут файл, инициализируют модель и проверят ее на тестовом множестве



## Шаг 0. Заберем данные из файла (пример работы с Pandas)

In [ ]:
import pandas as pd
import requests

orig_url='https://drive.google.com/file/d/1iIkvmDLb3zUn-iVpU1j7Zyp1C_Yv6tJx/view?usp=sharing'

# https://drive.google.com/file/d/1iIkvmDLb3zUn-iVpU1j7Zyp1C_Yv6tJx/view
file_id = orig_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
tenders = pd.read_csv(dwn_url,                    
                   encoding='utf-8',     # обязательно указываем кодировку utf-8! Иначе нам не распарсить! 
                     engine='python',        # не обязательно, но убирает один надоедливый WARNING от IPython
                     delimiter=',')           # обязательный параметр - разделитель!)
tenders.head()

,Name,Id,Region,Location,Buyer,TenderType,Placement,PublishedDate,StartingPrice,PurchaseArea
0,"""Выполнение работ по ремонту автомобильной дор...",50655430,Иркутская.обл,г. Иркутск,КОМИТЕТ ГОРОДСКОГО ОБУСТРОЙСТВА АДМИНИСТРАЦИИ ...,44-ФЗ,Конкурсы и аукционы,18.02.2021 14:45:10,13774127,"Строительство, ремонт и обслуживание дорог, мо..."
1,"""Поставка рамок алюминиевых для постера в Клин...",50653629,Москва.г,Город Москва,"ФКУЗ ""МСЧ МВД РОССИИ ПО Г. МОСКВЕ""",44-ФЗ,Закупки малого объема,18.02.2021 14:43:19,35000,"Оборудование и материалы для рекламы, изготовл..."
2,"""Ремонт проезда в микрорайоне №11, между домам...",50655159,Ямало-Ненецкий.АО,г. Губкинский,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""УПРАВЛЕНИЕ ...",44-ФЗ,Электронные аукционы,18.02.2021 14:41:07,900000,"Строительство, ремонт и обслуживание дорог, мо..."
3,"""Выполнение работ по ремонту дороги по ул. Ямк...",50655152,Ямало-Ненецкий.АО,"Приуральский район, село Аксарка",АДМИНИСТРАЦИЯ МУНИЦИПАЛЬНОГО ОБРАЗОВАНИЯ АКСАР...,44-ФЗ,Электронные аукционы,18.02.2021 14:40:54,10814664,"Строительство, ремонт и обслуживание дорог, мо..."
4,"""Поставка вывесок для нужд судов Иркутской обл...",50644717,Иркутская.обл,Иркутская область,УПРАВЛЕНИЕ СУДЕБНОГО ДЕПАРТАМЕНТА В ИРКУТСКОЙ ...,44-ФЗ,Электронные аукционы,18.02.2021 14:40:37,136500,"Оборудование и материалы для рекламы, изготовл..."


In [ ]:
# ЭТО НАЧАЛО ВАШЕЙ РАБОТЫ: загрузка файла CSV. 

import pandas as pd
# мы используем для работы с данными pandas
# создаем дата-фрейм из файла CSV

## ВНИМАНИЕ! Файл может некорректно отображаться при выводе! Если это произойдет, откройте его в текстовом редакторе
## и пересохраните в кодировке UTF8!!!!!!

tenders = pd.read_csv(
                     "/content/sample_data/tenders2_utf.csv", # путь к файлу!!! ВАШ ФАЙЛ ЛЕЖИТ В COLLAB! ЗАМЕНИТЕ!
                    #  encoding='utf-8',     # обязательно указываем кодировку utf-8! Иначе нам не распарсить! 
                     engine='python',        # не обязательно, но убирает один надоедливый WARNING от IPython
                     delimiter=','           # обязательный параметр - разделитель!
                    )

# Таким образом в нашем dataframe лежит весь набор данных целиком! Мы из него наберем нужные нам атрибуты,
# а также обучающее и тестовое подмножества.



# Выведем семпл из 5 случайных строк из DataFrame для того, чтобы проверить, все ли нормально прошло.

tenders.sample(5)

,Name,Id,Region,Location,Buyer,TenderType,Placement,PublishedDate,StartingPrice,PurchaseArea
13652,"""Выполнение работ по ремонту автомобильной дор...",49399457,Ханты-Мансийский Автономный округ - Югра.АО,г. Нижневартовск,"МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ""УПРАВЛЕНИЕ...",44-ФЗ,Электронные аукционы,08.12.2020,85167021,"Строительство, ремонт и обслуживание дорог, мо..."
7633,"""Механизированная очистка дорог от снега на те...",50050103,Калужская.обл,"Бабынинский район, село Сабуровщино",АДМИНИСТРАЦИЯ (ИСПОЛНИТЕЛЬНО-РАСПОРЯДИТЕЛЬНЫЙ ...,44-ФЗ,Закупки малого объема,20.01.2021,53650,"Строительство, ремонт и обслуживание дорог, мо..."
13611,"""Выполнение работ по содержанию автомобильной ...",49397125,Коми.Респ,"Удорский район, село Большая Пучкома; деревня ...","МУНИЦИПАЛЬНОЕ КАЗЁННОЕ УЧРЕЖДЕНИЕ ""УПРАВЛЕНИЕ ...",44-ФЗ,Электронные аукционы,08.12.2020,953748,"Строительство, ремонт и обслуживание дорог, мо..."
16578,"""Стенд уличный тактильный 1210х910 мм""",49173046,Москва.г,Город Москва,ГБПОУ ПТ №47,44-ФЗ,Закупки малого объема,30.11.2020,118250,"Оборудование и материалы для рекламы, изготовл..."
17650,"""Выполнение работ по ремонту подъездной и пеше...",49146253,Московская.обл,г. Видное,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЛЕНИНСКОГО ...,44-ФЗ,Закупки малого объема,27.11.2020,487338,"Строительство, ремонт и обслуживание дорог, мо..."


In [ ]:

# Нам нужна только часть полей. Те поля, которые отмечены в списке "Информационно", мы НЕ будем использовать в анализе. 

# Так мы укажем, что нам (возможно) потребуется. Минимум полей: Name (даннные для обучения) и PurchaseArea - разметка, 
# которую мы предсказываем

tenders[["Id", "Name", "TenderType", "Placement", "StartingPrice", "PurchaseArea"]].sample(5)

,Id,Name,TenderType,Placement,StartingPrice,PurchaseArea
4771,50276052,"""Изготовление таблички""",44-ФЗ,Закупки малого объема,166000,"Оборудование и материалы для рекламы, изготовл..."
13376,49447097,"""Закупка навигационных табличек для МБУК Серед...",44-ФЗ,Закупки малого объема,68300,"Оборудование и материалы для рекламы, изготовл..."
17246,49117933,"""Оказание услуг по содержанию внутрипоселковых...",44-ФЗ,Электронные аукционы,2645712,"Строительство, ремонт и обслуживание дорог, мо..."
5066,50260721,"""Изготовление информационных табличек""",Коммерческие,Нерегламентированные закупки,128490,"Оборудование и материалы для рекламы, изготовл..."
14872,49337733,"""Поставка набора информационных стендов (табли...",44-ФЗ,Закупки малого объема,97754,"Оборудование и материалы для рекламы, изготовл..."


## Шаг 1. Учимся отбирать те записи, у которых подходящая маркировка. То есть нет смешанных классификаторов. Таких записей достаточно много 

In [ ]:
# наши инструменты для преобразований:

# Напрямую обращаемся к колонке (атрибуту), например, берем колонку Name
# tenders.Name

# посмотреть уникальные значения указанного поля: 
#tenders[["PurchaseArea"]].drop_duplicates()

# взять строку по индексу:  
#tenders[["PurchaseArea"]].loc[[0]]

# отобрать строки по значению колонки:
tenders.loc[tenders["PurchaseArea"] == "Строительство, ремонт и обслуживание дорог, мостов, тоннелей и ЖД путей" ]

# наши (предполагаемые) целевые тематики:
areas = [
           "Строительство, ремонт и обслуживание дорог, мостов, тоннелей и ЖД путей",
           "Ремонт зданий и сооружений Строительство, ремонт и обслуживание дорог, мостов, тоннелей и ЖД путей",
           "Оборудование и материалы для рекламы, изготовление и монтаж (кроме полиграфической продукции)",
           "Готовые металлические изделия, Ограждения, Изделия ковки",
           "Материалы для строительства дорог, ЖД путей",
           "Строительство, ремонт и обслуживание дорог, мостов, тоннелей и ЖД путей Благоустройство и озеленение"          
    
]
# отобрать строки, где целевая колонка имеет значения, входящие в списку:
# мы делаем это, чтобы отсеять смешанные классификаторы
tenders[["Id", "Name", "TenderType", "Placement", "StartingPrice", "PurchaseArea"]].loc[tenders["PurchaseArea"].isin(areas)]


,Id,Name,TenderType,Placement,StartingPrice,PurchaseArea
0,50655430,"""Выполнение работ по ремонту автомобильной дор...",44-ФЗ,Конкурсы и аукционы,13774127,"Строительство, ремонт и обслуживание дорог, мо..."
1,50653629,"""Поставка рамок алюминиевых для постера в Клин...",44-ФЗ,Закупки малого объема,35000,"Оборудование и материалы для рекламы, изготовл..."
2,50655159,"""Ремонт проезда в микрорайоне №11, между домам...",44-ФЗ,Электронные аукционы,900000,"Строительство, ремонт и обслуживание дорог, мо..."
3,50655152,"""Выполнение работ по ремонту дороги по ул. Ямк...",44-ФЗ,Электронные аукционы,10814664,"Строительство, ремонт и обслуживание дорог, мо..."
4,50644717,"""Поставка вывесок для нужд судов Иркутской обл...",44-ФЗ,Электронные аукционы,136500,"Оборудование и материалы для рекламы, изготовл..."
...,...,...,...,...,...,...
20088,48935404,"""Выполнение работ по объекту: Устройство элеме...",44-ФЗ,Электронные аукционы,38631619,"Строительство, ремонт и обслуживание дорог, мо..."
20089,48935757,"""Поставка информационных табличек и стендов, в...",44-ФЗ,Электронные аукционы,91948,"Оборудование и материалы для рекламы, изготовл..."
20091,48934612,"""Поставка заграждений автомобильного проезда т...",44-ФЗ,Электронные аукционы,659885,"Готовые металлические изделия, Ограждения, Изд..."
20096,48935993,"""Поставка светодиодного экрана""",44-ФЗ,Электронные аукционы,1850000,"Оборудование и материалы для рекламы, изготовл..."


## Шаг 3. Готовим итоговый датасет для подачи в модель 

In [ ]:
# Делим на обучающее и тестовое множества

import numpy as np

# отбираем два поля в анализ, фильтруем интересные нам тематики

analyzed_tenders = tenders[["Name", "PurchaseArea"]].loc[tenders["PurchaseArea"].isin(areas)]

# генерируем массив случайных чисел от 0.0 до 1.0 длиной с наш датасет.

msk = np.random.rand(len(analyzed_tenders)) < 0.8

# делим датасет на два куска: число до 0.8 - обучающее, число 0.8 и выше - тренировочное множество.

tenders_train = analyzed_tenders[msk]
tenders_test = analyzed_tenders[~msk]

from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

dicts = {}

label.fit(tenders_train.PurchaseArea.drop_duplicates())

train_source_array = tenders_train.Name.apply(lambda Name: np.array(list(Name.split(' '))))

# Мы векторизуем текстовое поле с названием:
# - составляем словарь всех слов во всех строках датасета. 
# - получаем разреженную матрицу размерности M x N (M - число записей, N-число слов)  
#   где для каждого документа стоит 1, если слово встречается и 0, если слова нет. 

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_tenders_count = count_vect.fit_transform(tenders_train.Name)

X_tenders_count.shape

# Далее, мы конвертируем матрицу встречаемости в метрику  TF-IDF
# Таким образом мы часто встречающимся по документам словам низкий балл
# а характеристичным (они связаны с темой, указанной в маркировке) - высокий балл

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

tenders_train_tfidf = tfidf_transformer.fit_transform(X_tenders_count)

# Оценим, что у нас получается: Число строк (документов) и число слов в словаре (число значений TF-IDF)
print(tenders_train_tfidf.shape)



(8914, 14635)


## Шаг 4. Тренируем модели.

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

tenders_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                             alpha=1e-3, max_iter=10, tol=0.2, random_state=42)),
                        ])
_ = tenders_clf_svm.fit(tenders_train.Name, tenders_train.PurchaseArea)
predicted_svm = tenders_clf_svm.predict(tenders_test.PurchaseArea)
np.mean(predicted_svm == tenders_test.PurchaseArea)



0.8894865525672372

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier( random_state=42)

tenders_clf_rf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-rf',clf)])

_ = tenders_clf_rf.fit(tenders_train.Name, tenders_train.PurchaseArea)
predicted_rf = tenders_clf_rf.predict(tenders_test.PurchaseArea)
np.mean(predicted_rf == tenders_test.PurchaseArea)

0.8911985846970367

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV

clf2 = RandomForestClassifier(random_state=42)

tenders_clf_rf2 = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-rf',clf2)])

parameters = { 'clf-rf__n_estimators': [10,20,50,100,100,200, 500],
    'clf-rf__max_features': ['auto', 'sqrt', 'log2'],
    'clf-rf__max_depth' : [4,5,6,7,8,None],
    'clf-rf__criterion' :['gini', 'entropy']}

gs = GridSearchCV(tenders_clf_rf2, parameters, cv=2, n_jobs=-1, scoring='accuracy')
gs.fit(tenders_train.Name, tenders_train.PurchaseArea)
gs.best_score_

0.9335876149876599

## Шаг 5. Сохраняем обученную модель в нужные форматы данных и тестируем!

## Pickle

In [ ]:
import pickle
ss = pickle.dumps(gs.best_estimator_)

#Сохраняем файл
with open('/content/rf.pkl', 'wb') as pkl_file:
    p1 = pickle.dump(ss,pkl_file)

# Загружаем сохраненный файл
with open('/content/rf.pkl', 'rb') as pkl_file:
    p1 = pickle.load(pkl_file)
pipe_from_bytes = pickle.loads(p1)

# Прогноз
pipe_from_bytes.predict(tenders_test.loc[7:17,'Name'])

array(['Оборудование и материалы для рекламы, изготовление и монтаж (кроме полиграфической продукции)'],
      dtype=object)

## Joblib

In [ ]:
## joblib
from joblib import dump, load
#Сохраняем файл
dump(gs.best_estimator_, 'rf_joblib.joblib') 
# Загружаем сохраненный файл
rf_joblib = load('/content/rf_joblib.joblib') 
# Прогноз
rf_joblib.predict(tenders_test.loc[7:17,'Name'])

array(['Оборудование и материалы для рекламы, изготовление и монтаж (кроме полиграфической продукции)'],
      dtype=object)

## ONNX

In [ ]:
!pip install skl2onnx

     |████████████████████████████████| 194kB 5.2MB/s 
     |████████████████████████████████| 14.5MB 309kB/s 
     |████████████████████████████████| 71kB 8.1MB/s 


In [ ]:
# import onnxruntime as rt
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import StringTensorType

initial_type = [('input', StringTensorType([None, 1]))]
model_onnx = convert_sklearn(gs.best_estimator_, initial_types=initial_type)

with open("rf.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())



/usr/local/lib/python3.6/dist-packages/skl2onnx/operator_converters/text_vectoriser.py:179: UserWarning: Converter for TfidfVectorizer will use scikit-learn regular expression by default in version 1.6.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/skl2onnx/common/_container.py:569: UserWarning: Unable to find operator 'Tokenizer' in domain 'com.microsoft' in ONNX, op_version is forced to 1.
  op_type, domain))


In [ ]:
import onnxruntime as rt
import numpy

sess = rt.InferenceSession("/content/rf.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run( [label_name], {input_name: np.array(tenders_test.loc[7:17,'Name']).reshape(-1,1)})
print(pred_onx)

[array(['Оборудование и материалы для рекламы, изготовление и монтаж (кроме полиграфической продукции)'],
      dtype=object)]


In [ ]:
# Примеры выгрузки в файл. 
# Сохраняем модель в файл Model_N.тип где N - номер вашей группы, а тип = Pickle, joblib, onnx
# работу ведите в отдельной ячейке, создайте за этой!

## ниже примеры из лекции:

## Pickle

from sklearn import svm
from sklearn import datasets

clf = svm.SVC()
X, y= datasets.load_iris(return_X_y=True)
clf.fit(X, y)

import pickle
s = pickle.dumps(clf)
## добавьте сохранение в файл этого объекта!
clf_p = pickle.loads(s)
clf_p.predict(X[0:1])
y[0]

ss = pickle.dumps(tenders_clf_rf)

with open('hw2.pkl', 'wb') as pkl_file:
    p1 = pickle.dump(mydict,pkl_file)


## joblib
from joblib import dump, load

dump(clf, 'filename.joblib')

clf_j = load('filename.joblib') 

clf_j.predict(X[0:1])
y[0]

## ONNX 

rom sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)


from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())


import onnxruntime as rt
import numpy
sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]



In [ ]:
## Шаг 6. Загружаем полученную модель и проверяем ее. На тестовых данных.
## ниже примеры из лекции:

## используйте пример из лекции и загрузите последовательно 